In [19]:
# Install requirements
# !pip install -r requirements.txt

# Example code for querying geonetwork

Mostly from https://owslib.readthedocs.io/en/latest/usage.html#csw

In [20]:
import yaml
import owslib
from owslib.csw import CatalogueServiceWeb
from owslib.fes import PropertyIsEqualTo, PropertyIsLike, BBox

# Set up your username and password in a file creds.yml with the contents:
# username: xxxxx
# password: xxxxx

server = "https://catalogue.grumets.cat"
with open("./secrets.yml") as f:  creds = yaml.safe_load(f)

# Alternatively you can use a local catalogue using docker and the provided docker compose file
# server = "http://localhost:8080"
# creds = dict(username = 'geonetwork', password = 'geonetwork')

endpoint = server + "/geonetwork/srv/eng/csw?SERVICE=CSW&VERSION=2.0.2&REQUEST=GetCapabilities"

assert "username" in creds
assert "password" in creds
auth = owslib.util.Authentication(**creds)
csw = CatalogueServiceWeb(endpoint, timeout=60, 
                          auth = auth)

## Results contrained by keywords

In [21]:
query = PropertyIsEqualTo('csw:AnyText', 'water')
csw.getrecords2(constraints=[query], maxrecords=100)
csw.results

{'matches': 24, 'returned': 24, 'nextrecord': 0}

## Get all entries


In [22]:
csw.getrecords2(maxrecords=100)
csw.results

{'matches': 53, 'returned': 53, 'nextrecord': 0}

In [23]:
records = {}

for rec in csw.records:
    r = csw.records[rec]
    records[r.title] = r
    print(r.title)

# Choose the first record we found as an example
record = list(records.items())[0][1]

The Geoffrey's Tube Z3950 Server (Sample Record - Please Delete!)
Geoscience Australia's Open Day                             Photographs 26th August 2007
Localities in Victoria (VMADMIN.LOCALITY_POLYGON) - Comprehensive Elements
Hydrological Basins in Africa (Sample record, please remove!)
Plume Flow Air quality data for AD4GD
plume_example_points.shp
Surface water conductivity in river Spree, 2016
Citizen Science biodiversity observations in Farmer Cluster Mostviertler, Grünland, Austria
City Nature Challenge 2023 - biodiversity observations in Farmer Cluster Monte Pisano, Calci, Italy
PurpleAir Indoor Sensor Air Quality Data Sample for AD4GD (Sensor ID 157047)
IoT Sensor in Geneva
Continuous E.coli measurements in surface water
Citizen Science biodiversity observations in Farmer Cluster Monte Pisano, Calci, Italy
Groß Glienicker See- water level and water temperature
Hundekehlesee- water level and water temperature
Halensee- water level and water temperature
Orankesee- water level a

In [24]:
# Useful keys on the record object
[k for k in dir(record) if not k.startswith("_")]

['abstract',
 'accessrights',
 'alternative',
 'bbox',
 'bbox_wgs84',
 'contributor',
 'coverage',
 'created',
 'creator',
 'date',
 'format',
 'identifier',
 'identifiers',
 'ispartof',
 'issued',
 'language',
 'license',
 'modified',
 'publisher',
 'rdf',
 'references',
 'relation',
 'rights',
 'rightsholder',
 'source',
 'spatial',
 'subjects',
 'temporal',
 'title',
 'type',
 'uris',
 'xml']

## Pretty print the summary xml returned by a query

In [25]:
import xml.etree.ElementTree as ET

xml = ET.fromstring(record.xml)
print(ET.tostring(xml, encoding="unicode", method="xml"))

<ns0:SummaryRecord xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:ns0="http://www.opengis.net/cat/csw/2.0.2" xmlns:ns2="http://purl.org/dc/terms/">
      <dc:identifier>9f4c30af-d37f-4239-8b90-14eb7ea9d1d4</dc:identifier>
      <dc:title>The Geoffrey's Tube Z3950 Server (Sample Record - Please Delete!)</dc:title>
      <dc:type>service</dc:type>
      <ns2:modified>2010-03-24</ns2:modified>
      <ns2:abstract>This catalog is for registering all metadata records held by the Geofffrey's Tube Palace Hotel Ballroom.</ns2:abstract>
    </ns0:SummaryRecord>


## Get the full XML for a record

In [26]:
csw.getrecordbyid(id=[record.identifier])


for rec in csw.records:
    r = csw.records[rec]
    records[r.title] = r
    print(r.title)
    xml = ET.fromstring(r.xml)
    print(ET.tostring(xml, encoding="unicode", method="xml"))

The Geoffrey's Tube Z3950 Server (Sample Record - Please Delete!)
<ns0:Record xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:ns0="http://www.opengis.net/cat/csw/2.0.2" xmlns:ns2="http://purl.org/dc/terms/">
    <dc:identifier>9f4c30af-d37f-4239-8b90-14eb7ea9d1d4</dc:identifier>
    <dc:date>2009-10-15</dc:date>
    <dc:title>The Geoffrey's Tube Z3950 Server (Sample Record - Please Delete!)</dc:title>
    <dc:type>service</dc:type>
    <ns2:modified>2010-03-24</ns2:modified>
    <ns2:abstract>This catalog is for registering all metadata records held by the Geofffrey's Tube Palace Hotel Ballroom.</ns2:abstract>
    <dc:description>This catalog is for registering all metadata records held by the Geofffrey's Tube Palace Hotel Ballroom.</dc:description>
  </ns0:Record>


In [32]:
for title, record in records.items():
    print(record.title)
    for k in dir(record):
        if not k.startswith("_") and not k in ["xml", "abstract", "title"]: 
            val = getattr(record, k)
            if val: print(f"    {k}: {val}")
    print()

The Geoffrey's Tube Z3950 Server (Sample Record - Please Delete!)
    date: 2009-10-15
    identifier: 9f4c30af-d37f-4239-8b90-14eb7ea9d1d4
    identifiers: [{'scheme': None, 'identifier': '9f4c30af-d37f-4239-8b90-14eb7ea9d1d4'}]
    modified: 2010-03-24
    type: service

Geoscience Australia's Open Day                             Photographs 26th August 2007
    identifier: 59b3d999-ad30-4688-b705-6591d51f72cf
    identifiers: [{'scheme': None, 'identifier': '59b3d999-ad30-4688-b705-6591d51f72cf'}]
    modified: 2007-08-26
    type: collectionSession

Localities in Victoria (VMADMIN.LOCALITY_POLYGON) - Comprehensive Elements
    format: Most popular formats including ESRI shape, MapInfo Tab and Oracle Spatial
    identifier: fd1a659c-3c4c-4519-983a-ba2c5c992009
    identifiers: [{'scheme': None, 'identifier': 'fd1a659c-3c4c-4519-983a-ba2c5c992009'}]
    subjects: ['BOUNDARIES-Administrative', 'LAND-Ownership', 'boundaries']
    type: dataset

Hydrological Basins in Africa (Sample rec